# Better way 24. 객체를 범용으로 생성하려면 @classmethod 다형성을 이용하자

In [1]:
import os
from threading import Thread

* 파이썬에서는 **객체**가 다형성을 지원한 뿐만 아니라 **클래스**도 다형성을 지원한다.
* 다형성: 계층 구조에 속한 여러 클래스가 자체의 메서드를 독립적인 버전으로 구현하는 방식
  * 다형성을 이용하면 여러 클래스가 같은 인터페이스나 추상 기반 클래스는 충족하면서 다른 기능을 제공할 수 있다.
    * Better way 28. 커스텀 컨테이너 타입은 collections.abs 의 클래스를 상속받게 만들자
* 예시) MapReduce 구현을 작성할 때 입력 데이터를 표현하는 공통 클래스

In [2]:
class InputData(object):
    def read(self):
        raise NotImplementedError

In [3]:
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
        
    def read(self):
        return open(self.path).read()

* 원한다면 PathInputData 같은 *InputData 서브클래스*를 몇개든 만들 수 있음
  * 각 서브 클래스에서 *처리할 바이트 데이터를 반환하는 표준 인터페이스*인 read 를 구현할 것
  * 네트워크에서 데이터를 읽어오게 하거나, 데이터의 압축을 해제하게 만들거나...
* 예시) 표준 방식으로 입력 데이터를 처리하는 MapReduce 작업 클래스

In [4]:
class Worker(object):
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
        
    def map(self):
        raise NotImplementedError
        
    def reduce(self):
        raise NotImplementedError

* 예시) 적용하려는 특정 맵 리듀스 함수를 구현한 Worker의 서브 클래스, 줄바꿈 카운터

In [5]:
class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
        
    def reduce(self, other):
        self.result += other.result

* 잘 작동할 것 같지만, 커다란 문제에 직면하게 된다.
  * 이 코드 조각들을 무엇으로 *연결*할 것인가?
  * 적절히 인터페이스를 설계하고 추상화한 클래스들 --> 객체를 생성한 뒤에나 유용
  * 무엇으로 객체를 만들고 맵리듀스를 조율할까?

* 예시) 헬퍼 함수로 객체를 만들고 연결하기

In [6]:
def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))
    
def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers

* map 단계를 여러 스레드로 나눠서 Worker 인스턴스를 실행
  * Better way 37. 스레드를 블로킹 I/O 용으로 사용하고 병렬화용으로는 사용하지 말자
* 그런 다음 reduce 를 반복적으로 호출해서 결과를 최종값 하나로 합침

In [7]:
def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
        
    first, rest = workers[0], workers[1:]
    
    for worker in rest:
        first.reduce(worker)
        
    return first.result

* 마지막으로 단계별로 실행하려고 mapreduce 함수에서 모든 조각을 연결한다.

In [8]:
def mapreduce(data_dir):
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

* 테스트

In [9]:
from tempfile import TemporaryDirectory

def write_test_files(tmpdir):
    for i in range(10):
        with open(os.path.join(tmpdir, '%d.txt' % i), 'w') as f:
            for j in range(i):
                f.write('%d\n' % j)

In [10]:
with TemporaryDirectory() as tmpdir:
    write_test_files(tmpdir)
    result = mapreduce(tmpdir)

print('There are', result, 'lines')

There are 45 lines


* 무엇이 문제일까?
  * mapreduce 함수가 범용적이지 않음
    * 다른 InputData나 Workder 서브클래스를 작성한다면 genrate_inputs, create_workers, mapreduce 함수를 다시 작성해야 한다
* 범용적인 방법은?
  * 다른 언어에서는 생성자 다형성으로 해결
    * 각 InputData 서브 클래스에서 맵리듀스를 조율하는 헬퍼 메서드가 범용적으로 사용할 수 있는 특별한 생성자 <-
  * 근데 파이썬에는 단일 생성자 메서드 __init__ 만 허용함
    * 모든 InputData 서브클래스가 호환되는 생성자를 갖춰야 한다?? 터무니 없음

* @classmethod 다형성을 이용하자
  * 생성된 객체가 아니라 전체 클래스에 적용된다는 점만 빼면 InputData.read 에 사용한 인스턴스 메서드 다형성과 똑같다.
* 예시) 공통 인터페이스를 사용해 새 InputData 인스턴스를 생성하는 범용 클래스 메서드로 InputData 클래스를 확장한다.

In [11]:
class GenericInputData(object):
    def read(self):
        raise NotImplementedError
        
    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

* generate_inputs 메서드는 GenericInputData 를 구현하는 서브클래스가 해석할 설정 파라미터들을 담은 딕셔너리를 받는다.
* 입력 파일들을 얻어올 디렉터리를 config 로 알아낸다.

In [25]:
class PathInputData(GenericInputData):
    def __init__(self, path):
        self.path = path
    
    def read(self):
        return open(self.path).read()
    
    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

* 비슷하게, GenericWorker 클래스에 create_workers 헬퍼를 작성한다.
  * input_class 파라미터 (GenericInputData 서브클래스) 로 필요한 입력을 만들어냄
  * cls() 를 범용 생성자로 사용해서 GenericWorker 를 구현한 서브클래스의 인스턴스를 생성한다

In [19]:
class GenericWorker(object):
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
    
    def map(self):
        raise NotImplementedError
        
    def reduce(self, other):
        raise NotImplementedError
        
    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
            
        return workers

* 클래스 다형성
  * input_class.generate_inputs
  * create_workers 가 `__init__`메서드를 직접 사용하지 않고 cls 를 호출
* LineCountWorker 는 부모 클래스만 변경

In [20]:
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
        
    def reduce(self, other):
        self.result += other.result

* mapreduce 함수를 범용적으로 재작성

In [21]:
def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

* 테스트용 파일로 실행하면 같은 결과가 나옴
* 차이: mapreduce 함수가 범용적으로 동작하기 위해 더 많은 파라미터를 필요로 함

In [26]:
with TemporaryDirectory() as tmpdir:
    write_test_files(tmpdir)
    config = {'data_dir': tmpdir}
    result = mapreduce(LineCountWorker, PathInputData, config)

print('There are', result, 'lines')

There are 45 lines


### 핵심 정리
* 파이썬에서는 클래스별로 생성자를 한개만 만들 수 있다.
* 클래스에 필요한 다른 생성자를 정의하려면 @classmethod 를 사용하자
* 구체 서브클래스들을 만들고 연결하는 범용적인 방법을 제공하려면 클래스 메서드 다형성을 이용하자

# Better way 25. super 로 부모 클래스를 초기화하자

* 기존에는 자식 클래스에서 부모 클래스의 `__init__`메서드를 직접 호출하는 방식으로 부모 클래스를 초기화했다.

In [31]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        
class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

* 간단한 계층 구조에는 잘 동작하지만, 많은 경우 제대로 동작하지 못한다.
* 클래스가 다중 상속 (보통은 피해야...) 의 영향을 받는다면?
  * 부모 클래스의 `__init__` 메서드를 직접 호출하는 행위는 예기치 못한 동작을 일으킬 수 있다.
  * `__init__` 의 호출 순서가 모든 서브 클래스에 명시되어 있지 않기 때문.

In [29]:
class TimesTwo(object):
    def __init__(self):
        self.value *= 2
        
class PlusFive(object):
    def __init__(self):
        self.value += 5
        
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

* 이 클래스의 인스턴스를 생성하면 부모 클래스의 순서와 일치하는 결과가 만들어진다.

In [32]:
foo = OneWay(5)
print('First ordering is (5 * 2) + 5 =', foo.value)

First ordering is (5 * 2) + 5 = 15


* 같은 부모 클래스들을 다른 순서로 정의해보았다.
  * 부모 클래스를 정의한 순서와 부모 클래스 생성자를 호출하는 순서가 다름

In [34]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [35]:
bar = AnotherWay(5)
print('Second ordering still is', bar.value)

Second ordering still is 15


* 다이아몬드 상속 diamond inheritance
  * 서브 클래스가 계층 구조에서 같은 슈퍼클래스를 둔 서로 다른 두 클래스에서 상속받을 때 발생

In [37]:
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
        
class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 5
        
class ThisWay(TimesTwo, PlusFive):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)
        
foo = ThisWay(5)
print('Should be (5 * 5) + 2 = 27 but is', foo.value)

Should be (5 * 5) + 2 = 27 but is 10


* 두번째 부모 클래스의 생성자 `PlusTwo.__init__` 에서 다시 `MyBaseClass.__init__` 을 호출해서 self.value 를 5 로 리셋하기 때문

* 파이썬 2.2 부터 이런 문제를 해결하기 위해 `super`라는 내장 함수를 추가하고 메서드 해석 순서(MRO, method resolution order) 를 정의
* 메소드 해석 순서 MRO
  * 어떤 슈퍼 클래스부터 초기화하는지를 정한다.
    * 깊이 우선, 왼쪽 --> 오른쪽
  * 다이아몬드 계층 구조에 있는 공통 슈퍼 클래스를 단 한번만 실행하게 한다.

In [41]:
# 파이썬 2
class TimesFiveCorrect(MyBaseClass):
    def __init__(self, value):
        super(TimesFiveCorrect, self).__init__(value)
        self.value *= 5
        
class PlusTwoCorrect(MyBaseClass):
    def __init__(self, value):
        super(PlusTwoCorrect, self).__init__(value)
        self.value += 2
        
class GoodWay(TimesFiveCorrect, PlusTwoCorrect):
    def __init__(self, value):
        super(GoodWay, self).__init__(value)
    
foo = GoodWay(5)
print('Should be 5 * (5 + 2) = 35 and is', foo.value)

Should be 5 * (5 + 2) = 35 and is 35


* `TimesFiveCorrect.__init__` 을 먼저 실행할 수는 없을까? 결과가 27이 되도록?
  * 없음. MRO 가 정의하는 순서와 계산 순서가 일치함.

In [42]:
from pprint import pprint

pprint(GoodWay.mro())

[<class '__main__.GoodWay'>,
 <class '__main__.TimesFiveCorrect'>,
 <class '__main__.PlusTwoCorrect'>,
 <class '__main__.MyBaseClass'>,
 <class 'object'>]


* 위는 생성자 호출 순서이며, 모든 초기화 메서드는 `__init__`함수가 호출된 순서의 역순으로 실행된다.

In [45]:
# practice

class GoodWay2(PlusTwoCorrect, TimesFiveCorrect):
    def __init__(self, value):
        super(GoodWay2, self).__init__(value)
    
bar = GoodWay2(5)
print('Should be 2 + (5 * 5) = 27 and is', bar.value)

Should be 2 + (5 * 5) = 27 and is 27


In [46]:
pprint(GoodWay2.mro())

[<class '__main__.GoodWay2'>,
 <class '__main__.PlusTwoCorrect'>,
 <class '__main__.TimesFiveCorrect'>,
 <class '__main__.MyBaseClass'>,
 <class 'object'>]


* 파이썬 2 내장함수 super 의 문제점
  * 문법이 좀 장황하다. 현재 정의하는 클래스, self 객체, 메서드 이름 (보통 `__init__`)과 모든 인수를 설정해줘야 한다. 이런 생성 방법은 파이썬을 처음 접하는 프로그래머에게 혼란을 줄 수 있다.
  * super 를 호출하면서 현재 클래스의 이름을 지정해야 한다. 클래스의 이름을 변경(클래스의 계층 구조를 개선할 때 아주 흔히 하는!)하면 super 를 호출하는 모든 코드를 수정해야 한다.

* 파이썬 3 에서는 super 를 인수 없이 호출하면 `__class__` 와 self 를 인수로 넘겨서 호출한 것으로 처리!
* 파이썬 3 에서는 항상 super 를 쓰자.
  * 명확하고, 간결하고, 항상 제대로 동작하기 때문

In [47]:
class Explicit(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value * 2)
        
class Implicit(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 2)
        
assert Explicit(10).value == Implicit(10).value

* 파이썬 3 에서는 `__class__` 변수를 사용한 메서드에서 현재 클래스를 올바르게 참조하도록 해주므로 위 코드가 잘 동작.
  * 파이썬 2 는 `__class__` 가 정의되어 있지 않음

### 핵심 정리
* 파이썬의 표준 메서드 해석 순서(MRO) 는 슈퍼 클래스의 초기화 순서와 다이아몬드 상속 문제를 해결한다.
* 항상 내장함수 super 로 부모 클래스를 초기화하자.